In [1]:

import numpy as np

from collections import defaultdict
import copy

import sys
import os
import time
import datetime
import torch
import torchvision

import matplotlib.pyplot as plt
import torch.nn as nn



In [5]:
# due to warning from above, below steps imported torchvision
#print(torch.__version__)
#!conda install -c pytorch torchvision=0.14.1 -y
#import torchvision
#print(torchvision.__version__)

0.15.2a0


In [2]:
device ='cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [20]:

#load pretrained model
model = torchvision.models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features

#modify output layer for 2 classes - poisoned vs non-poisoned images
model.fc = torch.nn.Linear(num_ftrs, 2)

model.load_state_dict(torch.load("model_save/model.pth"))


/home/nbuser/anaconda3/envs/mypy3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

In [104]:
#freeze earlier layers - this is optional and it may work better to NOT freeze the earlier layers
for param in model.parameters():
    param.requires_grad = False
    for name, param in model.named_parameters():
        if "fc" in name:
            param.requires_grad = True 


In [21]:
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# comment out this or cifar as needed
#trainset = torchvision.datasets.ImageFolder(root="data/split/train",transform=transform)
#trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

#testset = torchvision.datasets.ImageFolder(root="data/split/train",transform=transform)
#testloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

cifar10 = torchvision.datasets.CIFAR10(root='./data/clean', train=True, download=False, transform=transform)

# relabel cifar10 to make frogs 0 and trucks 1
new_labels = []
for label in cifar10.targets:
    if label == 6:
        new_labels.append(0)
    elif label == 9:
        new_labels.append(1)
    else:
        new_labels.append(-1) #everything else becomes -1
cifar10.targets = new_labels

# create the dataset to then make the dataloader
frog_indices = [i for i, label in enumerate(cifar10.targets) if label == 0]  
truck_indices = [i for i, label in enumerate(cifar10.targets) if label == 1]  
frog_truck_indices = frog_indices + truck_indices
frog_truck_dataset = torch.utils.data.Subset(cifar10, frog_truck_indices)
trainloader = torch.utils.data.DataLoader(frog_truck_dataset, batch_size=32, shuffle=True, num_workers=2)

cifar10_test = torchvision.datasets.CIFAR10(root='./data/clean', train=False, download=False, transform=transform)

test_labels = []
for label in cifar10_test.targets:
    if label == 6:
        test_labels.append(0)
    elif label == 9:
        test_labels.append(1)
    else:
        test_labels.append(-1)
cifar10_test.targets = test_labels

frog_test_indices = [i for i, label in enumerate(cifar10_test.targets) if label == 0]
truck_test_indices = [i for i, label in enumerate(cifar10_test.targets) if label == 1]
frog_truck_test_indices = frog_test_indices + truck_test_indices
frog_truck_test_dataset = torch.utils.data.Subset(cifar10_test, frog_truck_test_indices)

testloader = torch.utils.data.DataLoader(frog_truck_test_dataset, batch_size=32, shuffle=False, num_workers=2)

EPOCHS=40
BATCH_SIZE = 32
TOTAL_SIZE = 9800
#TOTAL_TEST_SIZE = 
STEPS_PER_EPOCH = TOTAL_SIZE // BATCH_SIZE

TOTAL_TEST_SIZE = 200 
STEPS_PER_TEST_EPOCH = TOTAL_TEST_SIZE // BATCH_SIZE
#testset = torchvision.datasets.CIFAR10(root='./data/clean', train=False, download=False, transform=transform)
#testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

In [6]:
trainset

NameError: name 'trainset' is not defined

In [7]:
trainset.classes

['frog', 'truck']

In [7]:
# note, above two really only work if not using cifar
frog_count = sum(cifar10.targets[i] == 0 for i in frog_truck_indices)
truck_count = sum(cifar10.targets[i] == 1 for i in frog_truck_indices)

print(f"Frog count: {frog_count}, Truck count: {truck_count}")

Frog count: 5000, Truck count: 5000


In [8]:
model.train()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [22]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)  # Sending model to device
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=0.0007
)  # lr should be kept low so that the pre-trained weights don't change easily

In [108]:
def get_test():
    test_loss = []
    correct = 0
    incorrect = 0

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    for batch_idx, (data, target) in enumerate(testloader):
        if batch_idx == STEPS_PER_TEST_EPOCH:
            break

        # Model is used to predict the test data so we are switching off the gradient

        with torch.no_grad():

            data = data.to(device)
            target = target.long().to(device)
            output = model(data)
            criterion = nn.CrossEntropyLoss()
            loss = criterion(output, target)

            # Note that optimizer is not used because the model shouldn't learn the test dataset

            for i in range(BATCH_SIZE):
                a = []
                for j in output[i]:
                    a.append(float(j.detach()))

                pred = a.index(max(a))

                if pred == int(target[i]):
                    correct = correct + 1

                else:
                    incorrect = incorrect + 1

        test_loss.append(float(loss.detach()))
    print("CORRECT: " + str(correct), "INCORRECT: " + str(incorrect),"TEST ACCURACY: "+str(correct/(correct+incorrect)))
    return (
            correct/(incorrect+correct),
            sum(test_loss)/len(test_loss),
    )

In [109]:
acc_ , loss_ = get_test()
print("ACCURACY AND LOSS BEFORE TUNING")
print("ACCURACY : "+str(acc_),"LOSS : "+str(loss_))

CORRECT: 192 INCORRECT: 0 TEST ACCURACY: 1.0
ACCURACY AND LOSS BEFORE TUNING
ACCURACY : 1.0 LOSS : 0.00025489663797391887


In [10]:
torch.save(model.state_dict(), "model_save/model2.pth")

In [110]:
avg_test_loss_history = []
avg_test_accuracy_history = []
avg_train_loss_history = []
avg_train_accuracy_history = []


loss_history = []
accuracy_history = []

new_best = 0

for i in range(EPOCHS):

    start = time.time()
    print(
        "-----------------------EPOCH "
        + str(i)
        + " -----------------------------------"
    )
    for batch_idx, (data, target) in enumerate(trainloader):
        if batch_idx == STEPS_PER_EPOCH:
            break
        optimizer.zero_grad()  # Resetting gradients after each optimizations
        # Sending input , target to device
        data = data.to(device) 
        target = target.to(device)
        output = model(data)
        loss = criterion(output, target.reshape((BATCH_SIZE,)).long())
        loss_history.append(loss.detach())
        # The loss variable has gradient attached to it so we are removing it so that it can be used to plot graphs
        loss.backward()
        optimizer.step()  # Optimizing the model

        # Checking train accuracy

        correct = 0
        incorrect = 0
        for p in range(BATCH_SIZE):
            a = []
            for j in output[p]:
                a.append(float(j.detach()))

            pred = a.index(max(a))

            if pred == int(target[p]):
                correct = correct + 1

            else:

                incorrect = incorrect + 1

        print(
            "\r EPOCH "
            + str(i)
            + " MINIBATCH: "
            + str(batch_idx)
            + "/"
            + str(STEPS_PER_EPOCH)
            + " LOSS: "
            + str(loss_history[-1]),
            end = ""
            
        )
        
        accuracy_history.append(correct/(correct+incorrect))

    end = time.time()
    print(
        " \n EPOCH "
        + str(i)
        + " LOSS "
        + str(sum(loss_history[-STEPS_PER_EPOCH:]) / STEPS_PER_EPOCH)
        + " ETA: "
        + str(end - start)
        + " \n MAX LOSS: "
        + str(max(loss_history[-STEPS_PER_EPOCH:]))
        + " MIN LOSS: "
        + str(min(loss_history[-STEPS_PER_EPOCH:]))
        + " TRAIN ACCURACY: "
        + str(sum(accuracy_history[-STEPS_PER_EPOCH:]) / STEPS_PER_EPOCH)
    )
    
    avg_train_loss_history.append(sum(loss_history[-STEPS_PER_EPOCH:]) / STEPS_PER_EPOCH)
    avg_train_accuracy_history.append(sum(accuracy_history[-STEPS_PER_EPOCH:]) / STEPS_PER_EPOCH)

    test_acc , test_loss  = get_test()
    
    avg_test_accuracy_history.append(test_acc)
    avg_train_loss_history.append(test_loss)
    
    if test_acc>=new_best: 
        new_best = test_acc
        torch.save(model.state_dict(), "model_save/model.pth") # Saving our best model


-----------------------EPOCH 0 -----------------------------------
 EPOCH 0 MINIBATCH: 305/306 LOSS: tensor(3.0286e-06, device='cuda:0') 
 EPOCH 0 LOSS tensor(0.0005, device='cuda:0') ETA: 2.416653633117676 
 MAX LOSS: tensor(0.0242, device='cuda:0') MIN LOSS: tensor(2.6077e-08, device='cuda:0') TRAIN ACCURACY: 0.9997957516339869
CORRECT: 191 INCORRECT: 1 TEST ACCURACY: 0.9947916666666666
-----------------------EPOCH 1 -----------------------------------
 EPOCH 1 MINIBATCH: 305/306 LOSS: tensor(3.9784e-06, device='cuda:0') 
 EPOCH 1 LOSS tensor(0.0002, device='cuda:0') ETA: 2.4780585765838623 
 MAX LOSS: tensor(0.0226, device='cuda:0') MIN LOSS: tensor(0., device='cuda:0') TRAIN ACCURACY: 0.9998978758169934
CORRECT: 191 INCORRECT: 1 TEST ACCURACY: 0.9947916666666666
-----------------------EPOCH 2 -----------------------------------
 EPOCH 2 MINIBATCH: 305/306 LOSS: tensor(0.0002, device='cuda:0') 0')
 EPOCH 2 LOSS tensor(0.0012, device='cuda:0') ETA: 2.333292007446289 
 MAX LOSS: tenso

In [13]:
# autoencoder, works only on clean data
import torch.optim as optim

# define an autoencoder, downsampling twice before upsampling back to original
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()  # output in range [0,1]
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

autoencoder = Autoencoder()
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.0007)

# training
num_epochs = 30
for epoch in range(num_epochs):
    for images, _ in trainloader:  # labels aren't used
        outputs = autoencoder(images)
        loss = criterion(outputs, images)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

print("Calculating reconstruction error threshold...")
reconstruction_errors = []
for images, _ in trainloader:  # use clean training data
    for image in images:
        with torch.no_grad():
            reconstructed = autoencoder(image.unsqueeze(0))
            error = torch.mean((image - reconstructed.squeeze()) ** 2).item()
            reconstruction_errors.append(error)

mean_error = np.mean(reconstruction_errors)
std_dev = np.std(reconstruction_errors)
threshold = mean_error + 3 * std_dev  # 99% confidence
print(f"Calculated threshold: {threshold}")


Epoch 1/30, Loss: 0.1799446940422058
Epoch 2/30, Loss: 0.19608958065509796
Epoch 3/30, Loss: 0.18392331898212433
Epoch 4/30, Loss: 0.23482763767242432
Epoch 5/30, Loss: 0.13934367895126343
Epoch 6/30, Loss: 0.18541790544986725
Epoch 7/30, Loss: 0.15175534784793854
Epoch 8/30, Loss: 0.17478160560131073
Epoch 9/30, Loss: 0.19726961851119995
Epoch 10/30, Loss: 0.15457020699977875
Epoch 11/30, Loss: 0.13885794579982758
Epoch 12/30, Loss: 0.1480586677789688
Epoch 13/30, Loss: 0.1342870146036148
Epoch 14/30, Loss: 0.14605091512203217
Epoch 15/30, Loss: 0.13792386651039124
Epoch 16/30, Loss: 0.13033649325370789
Epoch 17/30, Loss: 0.15555238723754883
Epoch 18/30, Loss: 0.17800016701221466
Epoch 19/30, Loss: 0.18443433940410614
Epoch 20/30, Loss: 0.19879984855651855
Epoch 21/30, Loss: 0.09065332263708115
Epoch 22/30, Loss: 0.20447546243667603
Epoch 23/30, Loss: 0.15304066240787506
Epoch 24/30, Loss: 0.19165651500225067
Epoch 25/30, Loss: 0.1483944058418274
Epoch 26/30, Loss: 0.12879560887813568

In [111]:
print("ACCURACY : ",new_best)

ACCURACY :  1.0


In [14]:
plt.plot(avg_train_accuracy_history , label = "Train")
plt.plot(avg_test_accuracy_history , label = "Test")
plt.title('ACCURACY PER EPOCH')
plt.xlabel("EPOCHS")
plt.ylabel("ACCURACY")
plt.legend

NameError: name 'avg_train_accuracy_history' is not defined

In [23]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image

class CustomTestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = [os.path.join(root_dir, fname) for fname in os.listdir(root_dir) if fname.endswith(('.png', '.jpg', '.jpeg'))]
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path)
        
        if self.transform:
            image = self.transform(image)
        
        return image, img_path  

transform = transforms.Compose([
    transforms.ToTensor(),
])

testset = CustomTestDataset(root_dir='data/allfrog/frog', transform=transform)
testloader = DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

In [16]:
# in case of model mismatch error
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [24]:
# now with the model trained, run the model against the primary poisoned data
model.eval() 

predictions = []
image_paths = []
with torch.no_grad():
    for images, paths in testloader:
        images = images.to(device) # ensure images are on the device too
        
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)  # Get the predicted class (0 for frog, 1 for truck)
        
        predictions.extend(predicted.tolist())
        image_paths.extend(paths)

# Map predictions to 'good' (0) and 'bad' (1), then sort them
result_labels = ['good' if pred == 0 else 'bad' for pred in predictions]
sorted_paths, sorted_labels = zip(*sorted(zip(image_paths, result_labels)))

# Write sorted labels to the labels.txt file
with open("labels.txt", 'w') as f:
    for label in sorted_labels:
        f.write(f"{label}\n")

print(f"Predictions saved to labels.txt") # Confirmation that the predictions have finished


Predictions saved to labels.txt


In [18]:
# evaluate allfrog with autoencoder
model.eval()

def detect_anomaly(image, threshold):
    with torch.no_grad():
        reconstructed = autoencoder(image.unsqueeze(0))
        error = torch.mean((image - reconstructed.squeeze()) ** 2).item()
        return error > threshold, error

print("Detecting anomalies in the allfrog dataset...")
results = []

for images, paths in testloader:  
    for i, image in enumerate(images):
        is_anomaly, _ = detect_anomaly(image, threshold)
        label = "bad" if is_anomaly else "good" # good for frogs, bad for not frogs
        results.append((paths[i], label))

results.sort(key=lambda x: x[0]) # sorts by file names

with open("labels.txt", "w") as f:
    for _, label in results:
        f.write(f"{label}\n")

print(f"Anomalies detection complete, results saved to label.txt")

Detecting anomalies in the allfrog dataset...
Anomalies detection complete, results saved to label.txt


In [25]:
# print out some result data for easy analysis
with open("labels.txt", 'r') as f:
    lines = f.readlines()

labels = [line.strip() for line in lines]

good_count = labels.count("good")
bad_count = labels.count("bad")

results = f"Spread of labels:\nGood: {good_count}\nBad: {bad_count}\n"


total_poisoned_frogs = 500
correct_poisoned_frogs = bad_count  # assuming all predictions are correct

correct_poisoned_frog_percentage = (correct_poisoned_frogs / total_poisoned_frogs) * 100

results += f"Percentage of poisoned frogs identified: {correct_poisoned_frog_percentage:.2f}%\n"
print(results)

with open("results.txt", 'a') as r:
    r.write(results)
    r.write("\n" + "="*40 + "\n\n") # dividing line for more clarity

print(f"Results appended to results.txt")

Spread of labels:
Good: 5148
Bad: 352
Percentage of poisoned frogs identified: 70.40%

Results appended to results.txt
